In [24]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import pandas as pd
import os

In [41]:
file_basename='AUG-BVB_1st_half_ENG'
dirpath='/temp'
fileName=os.path.join(dirpath, file_basename+'.csv')

event_data=pd.read_csv(fileName, sep=',',header=0)

In [46]:
fileName=os.path.join(dirpath, file_basename+'.mp4')
for index, row in event_data.iterrows():
    #print row['secondOffset'], row['timeToCapture']
    if row['fileName'] == file_basename+'.wav':
        outFileName=os.path.join(dirpath, file_basename+str(index)+'.mp4')
        print 'Writing clip #',index,' from sec ',row['secondOffset'],' length ',row['timeToCapture']
        ffmpeg_extract_subclip(fileName, row['secondOffset'], row['secondOffset']+row['timeToCapture'], targetname=outFileName)
    else:
        print 'Skipping ', row['fileName']

MON_V_MONT_1H_ESP.wav
MON_V_MONT_1H_ESP.wav
MON_V_MONT_1H_ESP.wav
MON_V_MONT_1H_ESP.wav
MON_V_MONT_1H_ESP.wav
MON_V_MONT_1H_ESP.wav
MON_V_MONT_1H_ESP.wav
MON_V_MONT_2H_ESP.wav
MON_V_MONT_2H_ESP.wav
MON_V_MONT_2H_ESP.wav
MON_V_MONT_2H_ESP.wav
Writing clip # 11  from sec  180  length  5

[MoviePy] Running:
>>> ffmpeg -y -i /temp\AUG-BVB_1st_half_ENG.mp4 -ss 180.00 -t 5.00 -vcodec copy -acodec copy /temp\AUG-BVB_1st_half_ENG11.mp4
... command successful.
Writing clip # 12  from sec  305  length  5

[MoviePy] Running:
>>> ffmpeg -y -i /temp\AUG-BVB_1st_half_ENG.mp4 -ss 305.00 -t 5.00 -vcodec copy -acodec copy /temp\AUG-BVB_1st_half_ENG12.mp4
... command successful.
Writing clip # 13  from sec  320  length  5

[MoviePy] Running:
>>> ffmpeg -y -i /temp\AUG-BVB_1st_half_ENG.mp4 -ss 320.00 -t 5.00 -vcodec copy -acodec copy /temp\AUG-BVB_1st_half_ENG13.mp4
... command successful.
Writing clip # 14  from sec  367  length  6

[MoviePy] Running:
>>> ffmpeg -y -i /temp\AUG-BVB_1st_half_ENG.mp4 -ss 

# https://trac.ffmpeg.org/wiki/Concatenate